## Cell 1: Environment Setup

In [15]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import importlib

# Import and reload scanner module (ensures latest code changes are used)
import sp500_scanner
importlib.reload(sp500_scanner)
from sp500_scanner import get_sp500_tickers, scan_sp500, filter_buy_signals, create_excel_output, create_pdf_report, cleanup_old_scans

print("✓ Imports complete")
print(f"Ready to scan S&P 500 as of {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✓ Imports complete
Ready to scan S&P 500 as of 2026-01-05 22:38


## Cell 2: Run Full S&P 500 Scan

**This will take 5-10 minutes to complete.**

Parameters:
- `daily_bars=60` - 60 days of daily data
- `weekly_bars=52` - 52 weeks of weekly data  
- `concurrency=15` - 15 parallel threads (adjust if needed)

In [16]:
# Run the scan
start_time = datetime.now()
print(f"🔍 Starting S&P 500 scan at {start_time.strftime('%H:%M:%S')}...\n")

results_df = scan_sp500(daily_bars=60, weekly_bars=52, concurrency=15)

elapsed = (datetime.now() - start_time).total_seconds()
print(f"\n✓ Scan completed in {elapsed:.1f} seconds")

🔍 Starting S&P 500 scan at 22:38:51...

Fetching S&P 500 ticker list...
✓ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

🔍 Scanning 503 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 15 threads
✓ [1/503] ABBV   -> FULL HOLD + ADD      $220.18
✓ [5/503] ALB    -> FULL HOLD + ADD      $146.13
✓ [10/503] AFL    -> FULL HOLD + ADD      $112.19
✓ [18/503] GOOGL  -> FULL HOLD + ADD      $316.54
✓ [19/503] GOOG   -> FULL HOLD + ADD      $317.32
✓ [20/503] ALL    -> FULL HOLD + ADD      $205.57
✓ [25/503] AXP    -> FULL HOLD + ADD      $379.80
✓ [26/503] AMZN   -> FULL HOLD + ADD      $233.06
✓ [29/503] AIG    -> FULL HOLD + ADD      $84.38
✓ [32/503] AME    -> FULL HOLD + ADD      $211.31
✓ [33/503] APH    -> FULL HOLD + ADD      $139.88
✓ [34/503] AMGN   -> FULL HOLD + ADD      $320.72
✓ [35/503] APP    -> FULL HOLD + ADD      $632.91
✓ [36/503] AAPL   -> FULL HOLD + ADD      $267.26
✓ [37/503] APO    -> FULL HOLD + ADD      $149.76
✓ [39/503] ADI  

## Cell 3: Filter and Display Results by Confluence

In [17]:
# Filter for FULL HOLD + ADD signals
buy_df = filter_buy_signals(results_df, 'FULL HOLD + ADD')

print(f"{'='*80}")
print(f"🎯 FULL HOLD + ADD SIGNALS: {len(buy_df)} stocks")
print(f"{'='*80}\n")

# Breakdown by confluence
print("📊 Breakdown by Confluence:")
confluence_counts = buy_df['confluence'].value_counts()
for conf, count in confluence_counts.items():
    print(f"  {conf:12s}: {count:3d} stocks")

# Save to workspace scanner_results folder
results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
pdf_path = results_dir / f'scanner_report_{timestamp}.pdf'

# Generate Excel
create_excel_output(buy_df, excel_path)
print(f"\n✓ Excel saved to: {excel_path}")

# Generate PDF
print(f"\n📄 Creating PDF research document...")
create_pdf_report(buy_df, results_df, pdf_path, timestamp)
print(f"✓ PDF saved to: {pdf_path}")

# Cleanup old scans (keep 1 most recent, archive rest)
print("\n📁 Managing scan history...")
cleanup_old_scans(results_dir, max_files=1)

🎯 FULL HOLD + ADD SIGNALS: 201 stocks

📊 Breakdown by Confluence:
  WEAK        : 130 stocks
  EXTENDED    :  48 stocks
  BALANCED    :  23 stocks
✓ Excel file created: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2240.xlsx
  - All: 201 stocks
  - Balanced: 23 stocks
  - Extended: 48 stocks
  - Weak: 130 stocks

✓ Excel saved to: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2240.xlsx

📄 Creating PDF research document...
✓ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2240.pdf
  - 23 BALANCED stocks detailed
  - 48 EXTENDED stocks in watch list
✓ PDF saved to: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2240.pdf

📁 Managing scan history...
  📦 Archived: sp500_analysis_20260105_2226.xlsx
  📦 Archived: sp500_analysis_20260105_2207.xlsx
  📦 Archived: sp500_analysis_20260105_2103.xlsx
  📦 Archived: sp500_analysis_20260105_1945.xlsx
  📦 Archived: scanner_report_20260105

## Cell 4: Display BALANCED Stocks (Priority Buy List)

In [18]:
# Filter for BALANCED confluence only
balanced_df = buy_df[buy_df['confluence'] == 'BALANCED'].copy()
balanced_df = balanced_df.sort_values('ticker')

print(f"🎯 BALANCED STOCKS (Priority Buy Candidates): {len(balanced_df)}\n")

if not balanced_df.empty:
    # Display key columns
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in balanced_df.columns]
    
    print(balanced_df[available_cols].to_string(index=False))
    
    print(f"\n✅ These {len(balanced_df)} stocks have:")
    print("  ✓ FULL HOLD + ADD signal (Weekly P1 + Daily P1)")
    print("  ✓ BALANCED confluence (healthy technical setup)")
    print("  ✓ 'Enter on Dip' recommendation")
else:
    print("No BALANCED stocks found in this scan.")

🎯 BALANCED STOCKS (Priority Buy Candidates): 23

ticker  current_price recommendation        d50       d200        w10       w200
  AAPL         267.26   Enter on Dip 272.940846 231.964811 273.594434 191.501618
  AMGN         320.72   Enter on Dip 323.211371 294.279542 326.479382 261.091445
 BRK-B         498.52   Enter on Dip 498.280000 497.780849 499.692999 394.245849
  CBOE         252.11   Enter on Dip 251.715800 236.586574 252.869002 171.415324
   CEG         354.94   Enter on Dip 360.465316 313.334904 356.867160 172.377178
  CINF         161.96   Enter on Dip 162.195634 150.708210 163.078886 118.656421
  CSCO          75.58   Enter on Dip  75.461230  67.038620  75.954564  51.337482
    EW          84.17   Enter on Dip  84.378200  78.079450  85.105000  81.503850
   EXE         106.82   Enter on Dip 112.483108 106.300980 113.200696  85.873113
  GILD         118.30   Enter on Dip 122.147535 112.444059 122.189639  79.935261
   HSY         179.99   Enter on Dip 179.918182 174.241748 1

## Cell 5: Display EXTENDED Stocks (Watch List)

In [19]:
# Filter for EXTENDED confluence
extended_df = buy_df[buy_df['confluence'] == 'EXTENDED'].copy()
extended_df = extended_df.sort_values('ticker')

print(f"⏳ EXTENDED STOCKS (Wait for Pullback): {len(extended_df)}\n")

if not extended_df.empty:
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200']
    available_cols = [col for col in display_cols if col in extended_df.columns]
    
    # Show first 20
    print(extended_df[available_cols].head(20).to_string(index=False))
    
    if len(extended_df) > 20:
        print(f"\n... and {len(extended_df) - 20} more (see Excel file)")
    
    print(f"\n⚠️ These stocks have strong signals but are technically overbought.")
    print("  → Wait for pullback to support before entering")
else:
    print("No EXTENDED stocks found.")

⏳ EXTENDED STOCKS (Wait for Pullback): 48

ticker  current_price   recommendation        d50       d200
  ACGL          94.44 Wait for Support  92.024800  91.303800
   AIZ         240.42 Wait for Support 226.028422 206.790342
  AMZN         233.06 Wait for Support 231.828401 217.216100
   APH         139.88 Wait for Support 136.634449 107.054555
   AXP         379.80 Wait for Support 365.784266 316.681041
   CAH         205.45 Wait for Support 198.556530 162.191808
    CB         315.19 Wait for Support 297.325400 285.489081
   CCL          31.49 Wait for Support  27.741200  26.361200
   CME         275.06 Wait for Support 272.477866 269.333022
   CRH         128.31 Wait for Support 119.775201 105.144313
  CTSH          81.63 Wait for Support  77.400219  74.244442
   CVS          80.42 Wait for Support  78.769200  70.199836
    DG         138.93 Wait for Support 115.717600 105.667301
  EXPD         154.40 Wait for Support 141.716793 121.839361
  EXPE         287.47 Wait for Support 257

## Cell 6: Overall Signal Distribution

In [20]:
print("📊 Complete S&P 500 Signal Distribution:\n")
print(results_df['signal'].value_counts().to_string())

print(f"\n\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Total S&P 500 stocks analyzed: {len(results_df)}")
print(f"FULL HOLD + ADD signals: {len(buy_df)}")
print(f"  ├─ BALANCED (buy now): {len(buy_df[buy_df['confluence'] == 'BALANCED'])}")
print(f"  ├─ EXTENDED (watch): {len(buy_df[buy_df['confluence'] == 'EXTENDED'])}")
print(f"  └─ WEAK (skip): {len(buy_df[buy_df['confluence'] == 'WEAK'])}")
print(f"\nExcel file: {excel_path}")
print(f"PDF report: {pdf_path}")

📊 Complete S&P 500 Signal Distribution:

signal
FULL HOLD + ADD       201
HOLD MOST → REDUCE     79
FULL CASH / DEFEND     72
CASH                   52
REDUCE                 36
HOLD                   34
SCALE IN               21
LIGHT / CASH            7


Summary
Total S&P 500 stocks analyzed: 502
FULL HOLD + ADD signals: 201
  ├─ BALANCED (buy now): 23
  ├─ EXTENDED (watch): 48
  └─ WEAK (skip): 130

Excel file: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2240.xlsx
PDF report: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2240.pdf
